In [5]:
import threading
import time
import multiprocessing as mp
import sys
import random as rd

In [7]:
import threading
import time
import multiprocessing as mp
import sys
import random as rd

In [16]:
class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class puente():
    def __init__(self, number):
        self.number = number  + 1 #--!--       
        self.user = -1 #--!--      
        self.lock = threading.Condition()
        self.taken = False

    def take(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == True:
                sys.stdout.write("-------------Carro[%s] esperando por el puente----------\n" % (sentido))
                self.lock.wait()
            self.user = user
            self.taken = True
            sys.stdout.write("[%s] : Automóvil [%s] tomó el puente\n" % ( sentido, user))
            self.lock.notifyAll()

    def drop(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.user = -1 #--!--
            self.taken = False
            #sys.stdout.write("Automóvil[%s] dejó el puente [%s]\n" % (user, sentido))
            self.lock.notifyAll()
            

class AutomovilIzq (threading.Thread):
    def __init__(self, number, left, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1 #--!--   
        self.left = left
        #self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("<- Automóvil[%s] ESPERANDO\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.left.take(self.number, 'izquierdo')    
            time.sleep(1)     
            #self.right.take(self.number)    
            #time.sleep(1)                
            #sys.stdout.write("<- Automóvil[%s] PASAND0\n" % (self.number))
            #self.right.drop(self.number)    
            #time.sleep(1)
            self.left.drop(self.number, 'izquierdo')     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------- <-Automóvil[%s] terminó de PASAR--------------\n" % (self.number))
        
class AutomovilDer (threading.Thread):
    def __init__(self, number, right, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        #self.left = left
        self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("-> Automóvil[%s] ESPERANDO\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.right.take(self.number, 'dereho')    
            time.sleep(1)                 
            #sys.stdout.write("-> Automóvil[%s] PASAND0\n" % (self.number))
            self.right.drop(self.number, 'derecho')     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------- -> Automóvil[%s] terminó de PASAR--------------\n" % self.number)
 
def transit():
    n = int(input('Dime cuántos Autos : '))
    izq = rd.randint(1,n/2+1)
    der = n - izq
    print('derechos :',der, 'izquierdos :', izq)
    c = [puente(i) for i in range(n)]
    semaforo = Semaphore(n-1)
    sys.stdout.write("\n")
    AIzq = [AutomovilIzq(i, c[i],  semaforo) for i in range(izq)]
    ADer = [AutomovilDer(i, c[i],  semaforo) for i in range(der)]

    for i in range(izq):
        AIzq[i].start()
    for i in range(der):
        ADer[i].start()
        
    for j in range(izq):
        AIzq[i].join()
    for i in range(der):
        ADer[i].join()
    
transit()

Dime cuántos Autos : 6
derechos : 2 izquierdos : 4

<- Automóvil[1] ESPERANDO
<- Automóvil[2] ESPERANDO
<- Automóvil[3] ESPERANDO
<- Automóvil[4] ESPERANDO
-> Automóvil[1] ESPERANDO
-> Automóvil[2] ESPERANDO
[izquierdo] : Automóvil [2] tomó el puente
-------------Carro[dereho] esperando por el puente----------
[izquierdo] : Automóvil [4] tomó el puente
[izquierdo] : Automóvil [3] tomó el puente
[dereho] : Automóvil [1] tomó el puente
[dereho] : Automóvil [2] tomó el puente
---------------- <-Automóvil[2] terminó de PASAR--------------
---------------- <-Automóvil[3] terminó de PASAR--------------
---------------- <-Automóvil[4] terminó de PASAR--------------
---------------- -> Automóvil[1] terminó de PASAR--------------
[izquierdo] : Automóvil [1] tomó el puente
---------------- -> Automóvil[2] terminó de PASAR--------------
---------------- <-Automóvil[1] terminó de PASAR--------------


In [ ]:
### VERSIÓN SIN CHOQUES

class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class puente():
    def __init__(self, number):
        self.number = number  + 1 #--!--       
        self.user = -1 #--!--      
        self.lock = threading.Condition()
        self.taken = False

    def take(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == True:
                sys.stdout.write("-------------Carro[%s] esperando por el puente----------\n" % (sentido))
                self.lock.wait()
            self.user = user
            self.taken = True
            sys.stdout.write("[%s] : Automóvil [%s] se formó para el puente\n" % ( sentido, user))
            self.lock.notifyAll()

    def drop(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.user = -1 #--!--
            self.taken = False
            #sys.stdout.write("[%s] : Automóvil [%s] dejó el puente\n" % (sentido, user)) #--!--
            self.lock.notifyAll()
            

class AutomovilIzq (threading.Thread):
    def __init__(self, number, left, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1 #--!--   
        self.left = left
        #self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("<- Automóvil[%s] LLEGA\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.left.take(self.number, 'izquierdo')    
            time.sleep(1)     
            #self.right.take(self.number)    
            #time.sleep(1)                
            #sys.stdout.write("<- Automóvil[%s] PASAND0\n" % (self.number))
            #self.right.drop(self.number)    
            #time.sleep(1)
            self.left.drop(self.number, 'izquierdo')     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------- <-Automóvil[%s] terminó de PASAR--------------\n" % (self.number))
        
class AutomovilDer (threading.Thread):
    def __init__(self, number, right, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        #self.left = left
        self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("-> Automóvil[%s] LLEGA\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.right.take(self.number, 'derecho')    
            time.sleep(1)                 
            sys.stdout.write("-> Automóvil[%s] PASAND0\n" % (self.number))#--!--
            self.right.drop(self.number, 'derecho')     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------- -> Automóvil[%s] terminó de PASAR--------------\n" % self.number)
 
def transit():
    n = int(input('Dime cuántos Autos : '))
    izq = rd.randint(1,n//2+1)
    der = n - izq
    print('derechos :',der, 'izquierdos :', izq)
    c = [puente(i) for i in range(n)]
    semaforo = Semaphore(3)
    sys.stdout.write("\n")
    AIzq = [AutomovilIzq(i, c[i],  semaforo) for i in range(izq)]
    ADer = [AutomovilDer(i, c[i],  semaforo) for i in range(der)]

    for i in range(izq):
        AIzq[i].start()
    for i in range(der):
        ADer[i].start()
        
    for j in range(izq):
        AIzq[i].join()
    for i in range(der):
        ADer[i].join()
    
transit()

____
Segunds versión

In [54]:
### VERSIÓN SIN CHOQUES

class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class puente():
    def __init__(self, number):
        self.number = number  + 1 #--!--       
        self.user = -1 #--!--      
        self.lock = threading.Condition()
        self.taken = False

    def take(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == True:
                sys.stdout.write("-------------Carro[%s] esperando por el puente----------\n" % (sentido))
                self.lock.wait()
            self.user = user
            self.taken = True
            sys.stdout.write("[%s] : Automóvil [%s] se formó para el puente\n" % ( sentido, user))
            self.lock.notifyAll()

    def drop(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.user = -1 #--!--
            self.taken = False
            #sys.stdout.write("[%s] : Automóvil [%s] dejó el puente\n" % (sentido, user)) #--!--
            self.lock.notifyAll()
            

class AutomovilIzq (threading.Thread):
    def __init__(self, number, left, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1 #--!--   
        self.left = left
        #self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("-> Automóvil[%s] LLEGA\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.left.take(self.number, 'izquierdo')    
            time.sleep(1)     
            #self.right.take(self.number)    
            #time.sleep(1)                
            sys.stdout.write("-> Automóvil[%s] PASARÁ DESPUÉS \n" % (self.number))
            #self.right.drop(self.number)    
            #time.sleep(1)
            self.left.drop(self.number, 'izquierdo')     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------- -> Automóvil[%s] terminó de PASAR--------------\n" % (self.number))
        
class AutomovilDer (threading.Thread):
    def __init__(self, number, right, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        #self.left = left
        self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("<- Automóvil[%s] LLEGA\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.right.take(self.number, 'derecho')    
            time.sleep(1)                 
            sys.stdout.write("<- Automóvil[%s] PASARÁ DESPUÉS \n" % (self.number))#--!--
            self.right.drop(self.number, 'derecho')     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------- <- Automóvil[%s] terminó de PASAR--------------\n" % self.number)
 
def transit():
    n = int(input('Dime cuántos Autos : '))
    izq = rd.randint(1,n//2+1)
    der = n - izq
    print('derechos :',der, 'izquierdos :', izq)
    c = [puente(i) for i in range(n)]
    semaforo = Semaphore(3)
    sys.stdout.write("\n")
    AIzq = [AutomovilIzq(i, c[i],  semaforo) for i in range(izq)]
    ADer = [AutomovilDer(i, c[i],  semaforo) for i in range(der)]

    for i in range(izq):
        AIzq[i].start()
    for i in range(der):
        ADer[i].start()
        
    for j in range(izq):
        AIzq[i].join()
    for i in range(der):
        ADer[i].join()
    
transit()

Dime cuántos Autos : 9
derechos : 4 izquierdos : 5

-> Automóvil[1] LLEGA
-> Automóvil[2] LLEGA
-> Automóvil[3] LLEGA
-> Automóvil[4] LLEGA
-> Automóvil[5] LLEGA
<- Automóvil[1] LLEGA
<- Automóvil[2] LLEGA
<- Automóvil[3] LLEGA
<- Automóvil[4] LLEGA
[izquierdo] : Automóvil [1] se formó para el puente
[izquierdo] : Automóvil [2] se formó para el puente
[izquierdo] : Automóvil [5] se formó para el puente
-> Automóvil[1] PASARÁ DESPUÉS 
-> Automóvil[2] PASARÁ DESPUÉS 
-> Automóvil[5] PASARÁ DESPUÉS 
---------------- -> Automóvil[1] terminó de PASAR--------------
---------------- -> Automóvil[2] terminó de PASAR--------------
---------------- -> Automóvil[5] terminó de PASAR--------------
[izquierdo] : Automóvil [4] se formó para el puente
[izquierdo] : Automóvil [3] se formó para el puente
[derecho] : Automóvil [1] se formó para el puente
-> Automóvil[4] PASARÁ DESPUÉS 
-> Automóvil[3] PASARÁ DESPUÉS 
<- Automóvil[1] PASARÁ DESPUÉS 
---------------- -> Automóvil[4] terminó de PASAR-------

_______

In [17]:
class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class ChopStick():
    def __init__(self, number):
        self.number = number  + 1        
        self.user = -1      
        self.lock = threading.Condition()
        self.taken = False

    def take(self, user):         # used for synchronization
        with self.lock:
            while self.taken == True:
                self.lock.wait()
            self.user = user
            self.taken = True
            sys.stdout.write("Filósofo[%s] tomó el palillo[%s]\n" % (user, self.number))
            self.lock.notifyAll()

    def drop(self, user):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.user = -1
            self.taken = False
            sys.stdout.write("Filósofo[%s] dejó el palillo[%s]\n" % (user, self.number))
            self.lock.notifyAll()

class Philosopher (threading.Thread):
    def __init__(self, number, left, right, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        self.left = left
        self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("Filósofo[%s] PENSANDO\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.left.take(self.number)    
            time.sleep(1)     
            self.right.take(self.number)    
            time.sleep(1)                
            sys.stdout.write("Filósofo[%s] COMIENDO\n" % (self.number))
            self.right.drop(self.number)    
            time.sleep(1)
            self.left.drop(self.number)     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------Filósofo[%s] terminó de COMER--------------\n" % self.number)

def dinner():
    n = int(input('Dime cuántos Filósofos : '))
    c = [ChopStick(i) for i in range(n)]
    semaforo = Semaphore(2)
    sys.stdout.write("\n")
    p = [Philosopher(i, c[i], c[(i+1)%n], semaforo) for i in range(n)]

    for i in range(n):
        p[i].start()
        
    for j in range(n):
        p[i].join()
    
dinner()

KeyboardInterrupt: Interrupted by user